# Travail pour analyser notre dataset, notre modèle et nos résultats

#### On récupère le fichier Kaggle

In [23]:
from bigml.api import BigML
from pandas import read_csv
import json

df = read_csv('csv_file/train_full.csv')
api = BigML(project='project/5d94a454eba31d46690001d1')

#### Envoi sur BigML

##### Création source et dataset pour pouvoir travailler sur BigML

In [24]:
# Creation d'une source
source = api.create_source('csv_file/train_full.csv')
api.ok(source)
# Creation d'un dataset ( = source )
origin_dataset = api.create_dataset(source)
api.ok(origin_dataset)

True

#### Split du dataset Train/Val

In [27]:
train_set = api.create_dataset(
    origin_dataset, {"name": "train_set",
                     "sample_rate": 0.8, "seed": "my seed"})
validation_set = api.create_dataset(
    origin_dataset, {"name": "validation_set",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

#### Création fichier csv des 2 sets qui serviront pour l'amélioration continue
- cf notebook : GiveMeCredit_ContinuousImprovement

In [33]:
from bigml.api import BigML
from pandas import read_csv

api.ok(train_set)
api.download_dataset(train_set, filename='csv_file/train_set.csv')
api.ok(validation_set)
api.download_dataset(validation_set, filename='csv_file/validation_set.csv')

'csv_file/validation_set.csv'

#### Création d'un modele ensemble sur la partie train du split du dataset TrainFull (On precise quelle est la donnée que l'on recherche)

In [ ]:
ensemble = api.create_ensemble(train_set , {"objective_field" : "SeriousDlqin2yrs"})
print("OK")

#### Récupération des données d'évaluation du modèle sur validation_set

In [ ]:
import pandas

evaluation = api.create_evaluation(ensemble, validation_set)
api.ok(evaluation)
%store evaluation
eval = pandas.DataFrame(evaluation["object"]["result"]["model"]["confusion_matrix"], index=["F", "P"], columns=["F", "P"])
print(eval)

#### Prédiction du modèle sur le validation_set 

In [ ]:
batch_prediction_validation_set = api.create_batch_prediction(ensemble, validation_set,{"header": True, "all_fields": True,"prediction_name": "my_prediction", "probability": True, "probabilities": True})
api.ok(batch_prediction_validation_set)
api.download_batch_prediction(batch_prediction_validation_set,filename='BatchPrediction/Validation_set_prediction.csv')

#### Création de la matrice de confusion

##### Cette matrice sert à diviser les prédictions en 4 catégories :
- Vrai positifs : le modèle a prédit 1 et a eu raison
- Vrai négatifs : le modèle a prédit 0 et a eu raison
- Faux positifs : le modèle a prédit 1 et s'est trompé
- Faux négatifs : le modèle a prédit 0 et s'est trompé

In [ ]:
import pandas

prediction = pandas.read_csv('BatchPrediction/Validation_set_prediction.csv')
prediction.describe()

In [ ]:
def matrix_confusion_method(row):
    x = "TN"
    if row['SeriousDlqin2yrs'] == 1:
        if row['my_prediction'] == 1:
            x = "TP"
        else:
            x = "FN"
    else:
        if row['my_prediction'] == 1:
            x = "FP"
    return x

#### Affichage des valeurs de la matrice de confusion

In [ ]:
def get_matrix_confusion():
    prediction['Error'] = prediction.apply(matrix_confusion_method, axis=1)
    return prediction['Error'].value_counts()

matrix_confusion = get_matrix_confusion()
print(matrix_confusion)

#### Calcul de l'accuracy de notre modèle

In [ ]:
accuracy = (matrix_confusion['TP'] + matrix_confusion ['TN'])/300
print(accuracy)

## Etapes du travail d'analyse d'erreurs  :
- créer colonne "erreur" : i.e. colonne où on décrit si c'est un FN, FP, TN, TP
- analyser les plus grosses erreurs (probability très proche de 1)

#### Mise en forme de notre fichier pour analyse des erreurs les plus importantes

##### Ici, on a décidé de sélectionner les 100 plus grosses erreurs du modèle :
- C'est-à-dire que le modèle a fait une prédiction avec une probabilité très importante (ou avec un taux de confiance très fort) mais la prédiction est mauvaise

In [ ]:
filtered = prediction.loc[prediction['Error'].isin(["FN", "FP"])]
filtered = (filtered.nlargest(100, 'probability'))

filtered.to_csv('csv_file/100mistakes.csv')
filtered.describe()

In [ ]:
# import matplotlib.pyplot as plt

# %matplotlib inline

# table = [cost_threshold(n/10000) for n in range(0,10000)]
# plt.plot(table)